In [1]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
from datetime import datetime
import time

pd.options.mode.chained_assignment = None #to run loop quicker without warnings

# Preprocessing data

In [2]:
data = pd.read_csv('../data/helpdesk.csv')
data['CompleteTimestamp'] = pd.to_datetime(data['CompleteTimestamp'], errors='coerce')

In [3]:
data.head()

,CaseID,ActivityID,CompleteTimestamp
0,2,1,2012-04-03 16:55:38
1,2,8,2012-04-03 16:55:53
2,2,6,2012-04-05 17:15:52
3,3,1,2010-10-29 18:14:06
4,3,8,2010-11-04 01:16:11


In [4]:
data.shape

(13710, 3)

In [5]:
def calculateDuration(df):
    df['Duration'] = (df['CompleteTimestamp'] - df['CompleteTimestamp'].shift(1)).fillna(0)
    return df

def calculateCumDuration(df):
    df['CumDuration'] = (df['CompleteTimestamp'] - df['CompleteTimestamp'].iloc[0]).fillna(0) 
    #change df['CompleteTimestamp'][0] --> df['CompleteTimestamp'].iloc[0]
    return df

def calculateTimeSinceMidNight(x):
    x = str(x)
    x = time.strptime(x, "%Y-%m-%d %H:%M:%S")
    midnight = datetime.fromtimestamp(time.mktime(x)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(x))-midnight
    return timesincemidnight.seconds

def convert2seconds(x):
    x = int(x.total_seconds())
    return x

In [6]:
groupByCase = data.groupby(['CaseID'])

In [7]:
len(groupByCase)

3804

In [8]:
df = pd.DataFrame(columns=['CaseID', 'ActivityID', 'CompleteTimestamp', 'Duration', 'CumDuration', 'TimeSinceMidnight', 'WeekDay'])
#Loop all group and apply above functions
for case, group in groupByCase:
    group = calculateDuration(group)
    group = calculateCumDuration(group)
    group['Duration'] = group['Duration'].apply(convert2seconds)
    group['CumDuration'] = group['CumDuration'].apply(convert2seconds)
    group['TimeSinceMidnight'] = group['CompleteTimestamp'].apply(calculateTimeSinceMidNight)
    group['WeekDay'] = group['CompleteTimestamp'].dt.dayofweek
    
    df = df.append(group)

In [9]:
df.head(10)

,CaseID,ActivityID,CompleteTimestamp,Duration,CumDuration,TimeSinceMidnight,WeekDay
0,2.0,1.0,2012-04-03 16:55:38,0.0,0.0,60938.0,1.0
1,2.0,8.0,2012-04-03 16:55:53,15.0,15.0,60953.0,1.0
2,2.0,6.0,2012-04-05 17:15:52,173999.0,174014.0,62152.0,3.0
3,3.0,1.0,2010-10-29 18:14:06,0.0,0.0,65646.0,4.0
4,3.0,8.0,2010-11-04 01:16:11,457325.0,457325.0,4571.0,3.0
5,3.0,6.0,2010-11-04 01:21:17,306.0,457631.0,4877.0,3.0
6,4.0,1.0,2010-12-15 23:31:53,0.0,0.0,84713.0,2.0
7,4.0,8.0,2010-12-16 17:01:07,62954.0,62954.0,61267.0,3.0
8,4.0,6.0,2010-12-16 17:08:19,432.0,63386.0,61699.0,3.0
9,5.0,1.0,2012-04-03 21:08:32,0.0,0.0,76112.0,1.0


In [10]:
list(df)

['CaseID',
 'ActivityID',
 'CompleteTimestamp',
 'Duration',
 'CumDuration',
 'TimeSinceMidnight',
 'WeekDay']

In [11]:
for i in list(df):
    if i != 'CompleteTimestamp':
        df[i] = df[i].apply(int)

In [12]:
df.head(10)

,CaseID,ActivityID,CompleteTimestamp,Duration,CumDuration,TimeSinceMidnight,WeekDay
0,2,1,2012-04-03 16:55:38,0,0,60938,1
1,2,8,2012-04-03 16:55:53,15,15,60953,1
2,2,6,2012-04-05 17:15:52,173999,174014,62152,3
3,3,1,2010-10-29 18:14:06,0,0,65646,4
4,3,8,2010-11-04 01:16:11,457325,457325,4571,3
5,3,6,2010-11-04 01:21:17,306,457631,4877,3
6,4,1,2010-12-15 23:31:53,0,0,84713,2
7,4,8,2010-12-16 17:01:07,62954,62954,61267,3
8,4,6,2010-12-16 17:08:19,432,63386,61699,3
9,5,1,2012-04-03 21:08:32,0,0,76112,1


In [13]:
df.shape

(13710, 7)

In [14]:
df.to_csv('../input/new_data.csv', index=False)

In [15]:
df = pd.read_csv('../input/new_data.csv')

In [16]:
df.head()

,CaseID,ActivityID,CompleteTimestamp,Duration,CumDuration,TimeSinceMidnight,WeekDay
0,2,1,2012-04-03 16:55:38,0,0,60938,1
1,2,8,2012-04-03 16:55:53,15,15,60953,1
2,2,6,2012-04-05 17:15:52,173999,174014,62152,3
3,3,1,2010-10-29 18:14:06,0,0,65646,4
4,3,8,2010-11-04 01:16:11,457325,457325,4571,3


# Test

In [17]:
test = data[:10]
#data['CompleteTimestamp'] = data['CompleteTimestamp'].apply(parse)
test['CompleteTimestamp'] = pd.to_datetime(test['CompleteTimestamp'])

In [18]:
test

,CaseID,ActivityID,CompleteTimestamp
0,2,1,2012-04-03 16:55:38
1,2,8,2012-04-03 16:55:53
2,2,6,2012-04-05 17:15:52
3,3,1,2010-10-29 18:14:06
4,3,8,2010-11-04 01:16:11
5,3,6,2010-11-04 01:21:17
6,4,1,2010-12-15 23:31:53
7,4,8,2010-12-16 17:01:07
8,4,6,2010-12-16 17:08:19
9,5,1,2012-04-03 21:08:32


In [19]:
type(test['CompleteTimestamp'])

pandas.core.series.Series

In [20]:
test = calculateDuration(test)
test = calculateCumDuration(test)

In [21]:
test['Duration'] = test['Duration'].apply(convert2seconds)
test['CumDuration'] = test['CumDuration'].apply(convert2seconds)

In [22]:
test['Timefrommidnight'] = test['CompleteTimestamp'].apply(calculateTimeSinceMidNight)

In [23]:
test['WeekDay'] = test['CompleteTimestamp'].dt.dayofweek

In [24]:
test['CaseID'] = test['CaseID'].apply(float)

In [25]:
test

,CaseID,ActivityID,CompleteTimestamp,Duration,CumDuration,Timefrommidnight,WeekDay
0,2.0,1,2012-04-03 16:55:38,0,0,60938,1
1,2.0,8,2012-04-03 16:55:53,15,15,60953,1
2,2.0,6,2012-04-05 17:15:52,173999,174014,62152,3
3,3.0,1,2010-10-29 18:14:06,-45270106,-45096092,65646,4
4,3.0,8,2010-11-04 01:16:11,457325,-44638767,4571,3
5,3.0,6,2010-11-04 01:21:17,306,-44638461,4877,3
6,4.0,1,2010-12-15 23:31:53,3622236,-41016225,84713,2
7,4.0,8,2010-12-16 17:01:07,62954,-40953271,61267,3
8,4.0,6,2010-12-16 17:08:19,432,-40952839,61699,3
9,5.0,1,2012-04-03 21:08:32,40968013,15174,76112,1


Another way to get ```Duration```:

```python
import pandas as pd
from dateutil.parser import parse
data = pd.read_csv('../data/helpdesk.csv')
df = data[:10]
for i in range(0, len(df)-1):
    starting = parse(df.loc[i,'CompleteTimestamp'])
    ending = parse(df.loc[i+1,'CompleteTimestamp'])
    df.loc[i+1,'Duration'] = (ending-starting).total_seconds()

df.fillna(0)
```